<a href="https://colab.research.google.com/github/alex-gemeni/ML-playground/blob/main/Musika_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Musika: Generate Stereo Waveform Music in an Instant! 

In this notebook you will train a Musika system on your own music dataset by finetuning a pretrained model, and then share it on the [Musika Library](https://huggingface.co/musika)!

To train a model from scratch (for better results) take a look at the github repo: https://github.com/marcoppasini/musika

![Musika Logo](https://marcoppasini.github.io/logo.png)


In [ ]:
#@title Setup requirements

!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user

!git clone https://github.com/marcoppasini/musika.git
%cd musika
!pip install -r requirements.txt

Have a folder ready with a collection of audio files and either upload it to Colab or to your Google Drive (connect to it by executing the next cell). Be aware that single audio files need to be at least 50 seconds long to be properly preprocessed!

In [ ]:
#@title (Optional) Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Encode audio files to training samples

#@markdown Specify the path of the audio files to use preprocess for training
files_path = "/content/metal" #@param {type:"string"}

#@markdown (Optional) If you wish to save encodings to a different folder (maybe to your google drive to re-use them later) specify a path here
save_path = "/content/audio_encodings_folder" #@param {type:"string"}

!python musika_encode.py --files_path {files_path} --save_path {save_path}

In [ ]:
#@title Finetune a Musika model

#@markdown Specify the pretrained model to finetune
base_model = "misc" #@param ["misc", "techno"]

#@markdown Specify the path of the encoded training samples from the previous cell
train_path = "/content/audio_encodings_folder" #@param {type:"string"}

#@markdown (Optional) Number of epochs of the training process. The ideal number of epochs largely depends on the complexity of the training music domain: a number between 5 and 30 is recommended. One epochs always corresponds to about 10'000 iterations. 
epochs = 5 #@param {type:"integer"}

#@markdown (Optional) If you wish to use a different learning rate from the default one, change the following value
learning_rate = 0.00004 #@param {type:"number"}

#@markdown __IMPORTANT!__

#@markdown Click on the public URL to open a Gradio Interface. You can then test the model while it is training (the model used in the demo is updated after each epoch). Feel free to interrupt training earlier if the model produces good results! A tensorboard interface will be displayed: click on Settings, then tick Reload data to monitor training behavior.

%load_ext tensorboard
%tensorboard --logdir logs

!python musika_train.py --train_path {train_path} --load_path /content/musika/checkpoints/{base_model} --lr {learning_rate} --epochs {epochs} --share_gradio True

In [ ]:
#@title Test your model

#@markdown A Gradio interface will be created to test your trained model. Click on the public URL to access it.

#@markdown (Optional) By default, the last checkpoint in the musika/checkpoints folder will be selected. The path is in the form: /content/musika/checkpoints/MUSIKA_latlen_X_latdepth_X_sr_X_time_X-X/MUSIKA_iterations-Xk_losses-X-X-X
checkpoint_path = "" #@param {type:"string"}

import os

if(not checkpoint_path):
  directory = '/content/musika/checkpoints'
  exp_path = max([os.path.join(directory,d) for d in os.listdir(directory)], key=os.path.getmtime)
  checkpoint_path = max([os.path.join(exp_path,d) for d in os.listdir(exp_path)], key=os.path.getmtime)
print(f'Testing model in {checkpoint_path}')

!python musika_test.py --load_path {checkpoint_path} --share_gradio True

In [ ]:
#@title Generate and save samples

#@markdown Save a number of samples to a folder (for example on your Google Drive) and listen to them later.

#@markdown Number of samples to generate
num_samples = 10 #@param {type:"integer"}

#@markdown Length in seconds of each generated sample
seconds = 120 #@param {type:"integer"}

#@markdown Where generated samples are saved. If it does not exist, it will be automatically created.
save_path = "/content/generations" #@param {type:"string"}

#@markdown (Optional) By default, the last checkpoint in the musika/checkpoints folder will be selected. The path is in the form: /content/musika/checkpoints/MUSIKA_latlen_X_latdepth_X_sr_X_time_X-X/MUSIKA_iterations-Xk_losses-X-X-X
checkpoint_path = "" #@param {type:"string"}

import os

if(not checkpoint_path):
  directory = '/content/musika/checkpoints'
  exp_path = max([os.path.join(directory,d) for d in os.listdir(directory)], key=os.path.getmtime)
  checkpoint_path = max([os.path.join(exp_path,d) for d in os.listdir(exp_path)], key=os.path.getmtime)
print(f'Using model in {checkpoint_path}')

!python musika_generate.py --load_path {checkpoint_path} --num_samples {num_samples} --seconds {seconds} --save_path {save_path}

In [ ]:
#@title Login to the Huggingface Hub
#@markdown Input your Huggingface __write access__ token. To create one, go [here](https://huggingface.co/settings/tokens) and specify __write__ for the token role.

import sys
_ = (sys.path.append("/usr/local/lib/python3.9/site-packages"))

from huggingface_hub import notebook_login
!git config --global credential.helper store

notebook_login()

In [ ]:
#@title Save your model to the HuggingFace Hub

#@markdown You can either save your trained model to the public Musika models library, or save it privately.

#@markdown Specify a name for your Musika model
musika_model_name = "musika_metal" #@param {type:"string"}
#@markdown Where to save your model. You can allow other users to easily find and utilize your model by saving it in the Huggingface Musika organization, or you can save it as a private model on your account.
save_location = "musika_organization" #@param ["musika_organization", "private"]

#@markdown (Optional) By default, the last checkpoint in the musika/checkpoints folder will be selected. The path is in the form: /content/musika/checkpoints/MUSIKA_latlen_X_latdepth_X_sr_X_time_X-X/MUSIKA_iterations-Xk_losses-X-X-X
checkpoint_path = "" #@param {type:"string"}

# code inspired from https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/sd_dreambooth_training.ipynb

from slugify import slugify
from huggingface_hub import hf_hub_download
from huggingface_hub import HfApi, HfFolder, CommitOperationAdd
from huggingface_hub import create_repo
from IPython.display import display_markdown
import os

if(not checkpoint_path):
  directory = '/content/musika/checkpoints'
  exp_path = max([os.path.join(directory,d) for d in os.listdir(directory)], key=os.path.getmtime)
  checkpoint_path = max([os.path.join(exp_path,d) for d in os.listdir(exp_path)], key=os.path.getmtime)
print(f'Saving model in {checkpoint_path}')

api = HfApi()
your_username = api.whoami()["name"]

if(save_location == "musika_organization"):
  repo_id = f"musika/{slugify(musika_model_name)}"
  #Join the Musika organization if you aren't part of it already
  !curl -X POST -H 'Authorization: Bearer '$hf_token -H 'Content-Type: application/json' https://huggingface.co/organizations/musika/share/XjEqSfEgAwKDKTZyEzcyHvCbPaJYBcEAcz
else:
  repo_id = f"{your_username}/{slugify(musika_model_name)}"

with open(HfFolder.path_token, 'r') as fin: hf_token = fin.read();

readme_text = f'''---
license: mit
tags:
- audio
- music
- generation
- tensorflow
---

# Musika Model: {musika_model_name}
## Model provided by: {your_username}

Pretrained {musika_model_name} model for the [Musika system](https://github.com/marcoppasini/musika) for fast infinite waveform music generation.
Introduced in [this paper](https://arxiv.org/abs/2208.08706).

## How to use

You can generate music from this pretrained {musika_model_name} model using the notebook available [here](https://colab.research.google.com/drive/1HJWliBXPi-Xlx3gY8cjFI5-xaZgrTD7r).

### Model description

This pretrained GAN system consists of a ResNet-style generator and discriminator. During training, stability is controlled by adapting the strength of gradient penalty regularization on-the-fly. The gradient penalty weighting term is contained in *switch.npy*. The generator is conditioned on a latent coordinate system to produce samples of arbitrary length. The latent representations produced by the generator are then passed to a decoder which converts them into waveform audio.
The generator has a context window of about 12 seconds of audio.
'''
readme_file = open("README.md", "w")
readme_file.write(readme_text)
readme_file.close()
operations = [
  CommitOperationAdd(path_in_repo="README.md", path_or_fileobj="README.md"),
]
create_repo(repo_id, private=True, token=hf_token)
api.create_commit(
  repo_id=repo_id,
  operations=operations,
  commit_message=f"Upload {musika_model_name} readme",
  token=hf_token
)
api.upload_folder(
  folder_path=checkpoint_path,
  path_in_repo="",
  repo_id=repo_id,
  token=hf_token
)
display_markdown(f'''## Musika model successfully saved! See it [here](https://huggingface.co/{repo_id}). 
''', raw=True)